In [1]:
import pandas as pd
from preprocessing_tools import clean_text, lemmatize
from engine import Ranker
import os
import artm
from numpy import array
from topic_model import TopicModelWrapperARTM

# Load Data

In [2]:
path = '/home/sv9t/projects/comptech/data'
data_gaz = pd.read_csv(os.path.join(path, 'ready2viz/Политика.csv.gz') , compression='gzip')
data_class = pd.read_csv(os.path.join(path, 'classified/gazeta.csv.gz') , compression='gzip') 
data_raw = pd.read_csv(os.path.join(path, 'raw/gazeta.csv.gz'), compression='gzip') 
classnames = pd.read_json('/home/sv9t/projects/comptech/data/classnames_gazeta.json', typ='series')

/home/sv9t/py36venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
class_label = classnames[classnames == 'Политика'].index[0]
row_ids = data_class[data_class['rubric_preds'] == class_label]['row_id'].values
meta = data_raw['title'].values[row_ids]

# Load and Prepare Topic Model

In [3]:
tm = TopicModelWrapperARTM('./temp', 'query', 20)
tm.load_model('../data/tm/tm_gazeta_3.bin', '../data/tm/dictionary_tm_gazeta_3.txt')

In [4]:
query = 'америка президент сенатор доллар'

query = clean_text(query)
query = lemmatize(query)

print(query)

америка сенатор доллар


## Get Query Vector

In [5]:
tm.prepare_data([query])

In [6]:
theta = tm.transform()

In [7]:
theta

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
v_query = theta.values[0]

# Init Ranker and Get Nearest Vectors

In [11]:
colnames = [c for c in data_gaz.columns if c.startswith('topic_')]
ranker = Ranker(data_gaz[colnames].values, meta, 'cosine')
ixs = ranker.get_nearest(v_query)

In [12]:
ixs

array([54234, 54225, 57973, 10510, 78241, 74285,  5718, 94535, 91181,
       43494])

In [13]:
res = ranker.get_attributes(ixs)
for i, r in enumerate(res):
    print(i, r)

0 NYT: Brexit сделал Евросоюз слабее, а Россию – сильнее
1 Асад отметил неопытность последних президентов США во внешней политике 
2 Путин рассказал о поддержке в народе репрессивных мер МВД
3 Daily Beast: Сноуден сразу пожалел о вопросе Путину
4 РФ прослушивала секс по телефону Клинтона с Левински, утверждается в докладе Мюллера
5 МИД Канады приветствовал ликвидацию химического оружия в России
6 Правозащитники требуют от властей Ирана помиловать диссидента
7 Politico: ЕС и Британия могут обсудить Brexit на саммите в Брюсселе
8 Пушков: на конференции союзников не будет качественных сдвигов по борьбе с ИГ
9 Клинцевич: нагнуть Россию не получилось, решили вести диалог


In [14]:
def search(ranker, tm, query):
    query = clean_text(query)
    query = lemmatize(query)
    print('Processed query: ', query)
    if query == '':
        return "Query too short. Pass more then 3 words"
    tm.prepare_data([query])
    theta = tm.transform()
    print('Query vector:')
    print(theta)
    v_query = theta.values[0]
    ixs = ranker.get_nearest(v_query)
    return ranker.get_attributes(ixs)

In [15]:
search(ranker, tm, 'Нефть доллар рубль финансы')

Processed query:  нефть доллар рубль финансы


Query vector:
   topic_0  topic_1  topic_2  topic_3  topic_4  topic_5  topic_6   topic_7  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0  0.573178   

   topic_8  topic_9  topic_10  topic_11  topic_12  topic_13  topic_14  \
0      0.0      0.0       0.0       0.0  0.426822       0.0       0.0   

   topic_15  topic_16  topic_17  topic_18  topic_19  
0       0.0       0.0       0.0       0.0       0.0  


['Власти и повстанцы Колумбии подписали новое мировое соглашение',
 'МИД Венгрии заявил о новых ограничениях по использованию языка на Украине',
 'Секретарь «Петербургского диалога»: Баден-Вюртемберг не отказывался от форума, место еще не известно',
 'Путин: мы наелись революций в XX веке по горло',
 'Израиль поддержал «отважное» решение США о выходе из Совета ООН по правам человека',
 'Правительство Египта уходит в отставку',
 'Сикорский заявил, что не участвовал в переговорах Путина и Туска в 2008 году',
 'Песков: Путин и Мадуро обсудили ситуацию на нефтяном рынке',
 'Зюганов заявил о существовании доказательств обстрела Мариуполя силовиками',
 'Израиль вновь откроет два пограничных пункта на границе с сектором Газа']

In [16]:
search(ranker, tm, 'США КНР корея ядерное оружие')

Processed query:  сша кнр корея ядерный оружие


Query vector:
   topic_0  topic_1  topic_2  topic_3  topic_4  topic_5  topic_6  topic_7  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   topic_8  topic_9  topic_10  topic_11  topic_12  topic_13  topic_14  \
0      0.0      0.0       0.0       0.0       0.0       0.0       0.0   

   topic_15  topic_16  topic_17  topic_18  topic_19  
0       0.0       0.0       0.0       1.0       0.0  


['В Госдепе назвали цель вторжения в генконсульство России',
 'Порошенко призвал Европу создать коалицию против России',
 'Песков заявил о подготовке визита Путина в Финляндию',
 'Песков рассказал о взаимных интересах России и Европы',
 'Сандерс раскритиковал Клинтон за решение конфликтов с помощью военной силы',
 'Исламисты Мали угрожают настаивающей на интервенции Франции открыть для ее граждан «двери в ад»',
 '«Единая Россия»: депутат, сбивший члена КПРФ, от ЛДПР, а не из нашей партии',
 'Де Мистура доложит Совбезу ООН об итогах переговоров в Женеве',
 'Мособлизбирком: в Химках наблюдатели рвали одежду на членах комиссии, репетиция оранжевой революции',
 'СМИ сообщили, что переговоры между КНДР и Южной Кореей все еще продолжаются']

In [17]:
search(ranker, tm, 'киев украина майдан янукович')

Processed query:  киев украина майдан янукович


Query vector:
   topic_0  topic_1  topic_2  topic_3  topic_4  topic_5  topic_6  topic_7  \
0      0.0      0.0      1.0      0.0      0.0      0.0      0.0      0.0   

   topic_8  topic_9  topic_10  topic_11  topic_12  topic_13  topic_14  \
0      0.0      0.0       0.0       0.0       0.0       0.0       0.0   

   topic_15  topic_16  topic_17  topic_18  topic_19  
0       0.0       0.0       0.0       0.0       0.0  


['Трое ранены в ходе обстрела ополченцами пограничного отряда в Мариуполе',
 'Путин назвал бессмысленным желание увязать снятие санкций с решением кризиса на Украине',
 'В Болгарии начались первые обязательные выборы президента',
 'Путин: кризис на Украине — лишь формальный повод для санкций',
 'В партии Порошенко рассчитывают на отставку Яценюка в течение трех месяцев',
 'Мурат Зязиков будет баллотироваться на пост президента Ингушетии',
 '«Побеждать нокаутом»: тайный приказ Порошенко против Зеленского',
 'В Славянске из-за артобстрела загорелся завод',
 'Гуриев призвал Запад создать «план Маршалла» по восстановлению экономики России',
 'Нарушивший армяно-турецкую границу турок-пастух ранен пограничниками']

In [18]:
search(ranker, tm, 'киев украина майдан янукович США КНР корея ядерное оружие')

Processed query:  киев украина майдан янукович сша кнр корея ядерный оружие


Query vector:
   topic_0  topic_1   topic_2  topic_3  topic_4  topic_5  topic_6  topic_7  \
0      0.0      0.0  0.502446      0.0      0.0      0.0      0.0      0.0   

   topic_8  topic_9  topic_10  topic_11  topic_12  topic_13  topic_14  \
0      0.0      0.0       0.0       0.0       0.0       0.0       0.0   

   topic_15  topic_16  topic_17  topic_18  topic_19  
0       0.0       0.0       0.0  0.497554       0.0  


['Косачев: без США у КНДР и Южной Кореи вряд ли что-то получится сделать самостоятельно',
 'Прокуратура Украины проверяет запрос Грузии об экстрадиции Саакашвили',
 'Брат уходящего президента Южной Кореи приговорен к двум годам тюрьмы за коррупцию',
 'Восстановление здоровья Керри после перелома займет несколько месяцев',
 'Источник: Урлашова могут перевезти в Москву',
 'В Москве выступили за отмену односторонних санкций против Ирана',
 'Трамп рассказал о своих успехах на посту президента',
 'Захарова прокомментировала поговорку Пенса о русском медведе',
 'Клинтон побеждает в штатах Иллинойс, Нью Джерси, Виргиния',
 'Прокуратура Южной Кореи запросила 30 лет для экс-президента']

In [19]:
query = """
Москва объявила трех дипломатов Швеции, Польши и Германии, участвовавших в несанкционированных акциях 23 января, персонами нон-грата с обязательством покинуть Россию. Варшава, Стокгольм и Берлин предупредили об ответных мерах и ухудшении отношений с Россией. Госдума предупредила, что Россия не позволит «раздавать печеньки Госдепа» на улицах.
"""

In [20]:
search(ranker, tm, query)

Processed query:  москва объявить три дипломат швеция польша германия участвовать несанкционированный акция 23 январь персона нона грат обязательство покинуть россия варшава стокгольм берлин предупредить ответный мера ухудшение отношение россия госдума предупредить россия позволить раздавать печенек госдеп улица


Query vector:
   topic_0  topic_1   topic_2   topic_3   topic_4  topic_5  topic_6  topic_7  \
0      0.0      0.0  0.167779  0.102163  0.182403      0.0  0.36877      0.0   

   topic_8  topic_9  topic_10  topic_11  topic_12  topic_13  topic_14  \
0      0.0      0.0       0.0       0.0       0.0  0.140631       0.0   

   topic_15  topic_16  topic_17  topic_18  topic_19  
0       0.0       0.0  0.038254       0.0       0.0  


['В кабинете Никиты Белых идет обыск по делу, на причастность к которому проверяют Навального',
 'Пушков предлагает зеркально ответить на блокировку счетов «России сегодня»',
 'Генштаб: российская авиация наносит удары в Сирии только по объектам террористов',
 'Лавров и Керри обсуждают с Брахими и Пан Ги Муном урегулирование в Сирии',
 'Россия считает проект резолюции по Сребренице неприемлемым в нынешнем виде',
 'Главы государств — участников СНГ подписали заявление о борьбе с международным терроризмом',
 'Захарова прокомментировала инициативу венецианских политиков по Крыму',
 'Трамп встретился с Альбертом Гором',
 'Песков прокомментировал возможное участие женщины в выборах президента',
 'Хиллари Клинтон: Путин не извинился перед Обамой за опоздание на переговоры']

In [21]:
query = """
Мэр Якутска Сардана Авксентьева объявила о досрочном сложении полномочий в связи с состоянием здоровья.
До выборов ее обязанности будет исполнять первый замглавы Евгений Григорьев, его же Авскентьева призвала поддержать на выборах.
Сардана Авксентьева известна далеко за пределами Якутска и всего региона благодаря своим инициативам, 
направленным на экономию городского бюджета за счет сокращения затрат на автомобили для чиновников, 
продажи здания мэрии и отмены корпоративов.
"""

In [22]:
search(ranker, tm, query)

Processed query:  мэр якутск сардать авксентьев объявить досрочный сложение полномочие связь состояние здоровье до выбор обязанность исполнять замглавы евгений григорьев авскентьев призвать поддержать выборы сардать авксентьев известный предел якутск регион благодаря свой инициатива направить экономия городской бюджет счёт сокращение затрата автомобиль чиновник продажа здание мэрия отмена корпоратив


Query vector:
   topic_0  topic_1   topic_2  topic_3  topic_4  topic_5  topic_6  topic_7  \
0      0.0      0.0  0.107636      0.0      0.0      0.0      0.0      0.0   

   topic_8  topic_9  topic_10  topic_11  topic_12  topic_13  topic_14  \
0      0.0      0.0       0.0       0.0  0.780552       0.0       0.0   

   topic_15  topic_16  topic_17  topic_18  topic_19  
0  0.020407       0.0  0.091404       0.0       0.0  


['Правительственные войска Мали при поддержке сил Франции отбили у исламистов ключевой город Кона',
 'Премьер Нетаньяху не смог сформировать правительство Израиля, ему дали еще две недели',
 'Исландия планирует отказаться от вступления в Евросоюз',
 'Посол ЕС рассказал о сроках отмены виз для Украины',
 'Порошенко подтвердил готовность ЕС ввести санкции против России ',
 'Ольга Деева выбрана мэром Владимира',
 'Washington Times: закрытую переговорную Трампа и Путина в Хельсинки могли прослушивать финны',
 '«Мы поставим точку»: Путин и Абэ решили судьбу Курил',
 'Генпрокуратура Украины вызвала Порошенко на допрос',
 'Сенатор Совфеда Ананьев заработал больше, чем самый богатый человек в Кремле']